In [1]:
from __future__ import annotations
from pathlib import Path
import pandas as pd
import numpy as np


In [2]:
from pathlib import Path
import pandas as pd

def ensure_dir(p: Path):
    p.mkdir(parents=True, exist_ok=True)
    return p

def read_csv(path: str | Path, **kw) -> pd.DataFrame:
    return pd.read_csv(path, **kw)

def to_parquet(df, path: str | Path):
    path = Path(path)
    ensure_dir(path.parent)
    df.to_parquet(path, index=False)
    return str(path)


# Load metadata

In [3]:
metadata_1 = r"C:\bachkhoahanoi\2025.1\Recommend System\data\raw\sweettv-tv-program-recommender\export_arh_11-20-final.csv"
df_metadata_1 = read_csv(metadata_1)
df_metadata_1.head()

,channel_id,channel_title,start_time,duration,tv_show_title,tv_show_id,tv_show_category,tv_show_genre_1,tv_show_genre_2,tv_show_genre_3,year_of_production,director,actors
0,3,1+1,09.03.2020 06:00:00,6000,"'Жизнь известных людей', 2 сезон.",0,Другое,NaN,NaN,NaN,NaN,NaN,NaN
1,3,1+1,09.03.2020 07:40:00,5400,'ТСН. Неделя'.,2400480,Инфо,общество и политика,NaN,NaN,1997,NaN,NaN
2,3,1+1,09.03.2020 09:10:00,8400,"'Голос страны', 10 сезон, 8 эп. 'Бои'.",700475,Развлечения,музыка,NaN,NaN,2010,NaN,NaN
3,3,1+1,09.03.2020 11:30:00,28800,"Т/с 'По праву любви', 1-8 с.",0,Сериалы,NaN,NaN,NaN,NaN,NaN,NaN
4,3,1+1,09.03.2020 19:30:00,2700,ТСН.,2400480,Инфо,общество и политика,NaN,NaN,1997,NaN,NaN


In [4]:
metadata_2 = r"C:\bachkhoahanoi\2025.1\Recommend System\data\raw\sweettv-tv-program-recommender\export_arh_21-30-final.csv"
df_metadata_2 = read_csv(metadata_2)
df_metadata_2.head()

,channel_id,channel_title,start_time,duration,tv_show_title,tv_show_id,tv_show_category,tv_show_genre_1,tv_show_genre_2,tv_show_genre_3,year_of_production,director,actors
0,3,1+1,18.05.2020 06:30:00,1800,'Завтрак с 1+1'. Информационно-развлекательная...,20088,Развлечения,информация (комплексная),NaN,NaN,1996-,NaN,NaN
1,3,1+1,18.05.2020 07:00:00,600,ТСН.,2400480,Инфо,общество и политика,NaN,NaN,1997,NaN,NaN
2,3,1+1,18.05.2020 07:10:00,3000,'Завтрак с 1+1'. Информационно-развлекательная...,20088,Развлечения,информация (комплексная),NaN,NaN,1996-,NaN,NaN
3,3,1+1,18.05.2020 08:00:00,600,ТСН.,2400480,Инфо,общество и политика,NaN,NaN,1997,NaN,NaN
4,3,1+1,18.05.2020 08:10:00,3000,'Завтрак с 1+1'. Информационно-развлекательная...,20088,Развлечения,информация (комплексная),NaN,NaN,1996-,NaN,NaN


In [5]:
df_metadata_2.columns

Index(['channel_id', 'channel_title', 'start_time', 'duration',
       'tv_show_title', 'tv_show_id', 'tv_show_category', 'tv_show_genre_1',
       'tv_show_genre_2', 'tv_show_genre_3', 'year_of_production', 'director',
       'actors'],
      dtype='object')

In [6]:
# Metadata (export_arh_*)
def _parse_meta_start(series: pd.Series) -> pd.Series:
    """
    Parse start_time của metadata. Ưu tiên định dạng 'dd.mm.YYYY HH:MM:SS'.
    Fallback sang dayfirst=True nếu không khớp.
    """
    try:
        return pd.to_datetime(series, format="%d.%m.%Y %H:%M:%S", errors="raise")
    except Exception:
        return pd.to_datetime(series, dayfirst=True, errors="raise")

def load_meta(paths: list[str | Path]) -> pd.DataFrame:
    """
    Đọc và gộp các file export_arh_11-20-final.csv, export_arh_21-30-final.csv.
    - Đổi channel_id -> vsetv_id
    - Parse start_time (định dạng dd.mm.YYYY HH:MM:SS)
    - Tính prog_end = start_time + duration(s)
    - Trả về các cột cần cho interval join
    """
    if not paths:
        raise ValueError("`paths` for metadata is empty.")

    dfs = [read_csv(p) for p in paths]
    meta = pd.concat(dfs, ignore_index=True).copy()

    # Đổi tên cột & kiểm tra
    meta = meta.rename(columns={"channel_id": "vsetv_id"})
    required = {"vsetv_id", "start_time", "duration", "tv_show_id"}
    missing = required - set(meta.columns)
    if missing:
        raise ValueError(f"Missing columns in meta: {missing}")

    # Parse time + duration
    meta["start_time"] = _parse_meta_start(meta["start_time"])
    meta["duration"] = pd.to_numeric(meta["duration"], errors="coerce").fillna(0).astype(int)
    meta["prog_end"] = meta["start_time"] + pd.to_timedelta(meta["duration"], unit="s")

    # Cột text có thể vắng -> fill
    for c in ["tv_show_title", "tv_show_category", "tv_show_genre_1", "tv_show_genre_2", "tv_show_genre_3"]:
        if c not in meta.columns:
            meta[c] = ""

    # BẮT BUỘC: loại show không liên quan
    meta = meta[meta["tv_show_id"] != 0].copy() 
    
    cols = [
        "vsetv_id", "channel_title", "start_time", "prog_end","duration",
        "tv_show_title", "tv_show_id", "tv_show_category",
        "tv_show_genre_1", "tv_show_genre_2", "tv_show_genre_3",
        "year_of_production",	"director",	"actors"
    ]
    return meta[cols]


In [7]:
metadatas = []
metadatas.append(metadata_1)
metadatas.append(metadata_2)
meta = load_meta(metadatas)

In [8]:
meta.head

<bound method NDFrame.head of          vsetv_id        channel_title          start_time  \
1               3                  1+1 2020-03-09 07:40:00   
2               3                  1+1 2020-03-09 09:10:00   
4               3                  1+1 2020-03-09 19:30:00   
5               3                  1+1 2020-03-09 20:15:00   
6               3                  1+1 2020-03-10 00:10:00   
...           ...                  ...                 ...   
2147641      1528  ViP Premiere CEE HD 2020-07-26 23:05:00   
2147642      1528  ViP Premiere CEE HD 2020-07-27 00:40:00   
2147643      1528  ViP Premiere CEE HD 2020-07-27 02:50:00   
2147644      1528  ViP Premiere CEE HD 2020-07-27 04:15:00   
2147645      1528  ViP Premiere CEE HD 2020-07-27 05:50:00   

                   prog_end  duration  \
1       2020-03-09 09:10:00      5400   
2       2020-03-09 11:30:00      8400   
4       2020-03-09 20:15:00      2700   
5       2020-03-10 00:10:00     14100   
6       2020-03-10 0

# Load file logs

In [9]:
logs = r"C:\bachkhoahanoi\2025.1\Recommend System\data\raw\sweettv-tv-program-recommender\dataset11-30.csv"
df_logs = read_csv(logs)
df_logs.head()

,user_id,vsetv_id,start_time,stop_time,duraton
0,3398461054087191302,6,2020-03-09 00:00:02,2020-03-09 00:01:53,111
1,17226860011138219284,353,2020-03-09 00:00:02,2020-03-09 00:03:26,204
2,18417531283109304442,332,2020-03-09 00:00:05,2020-03-09 00:01:26,81
3,3677642666679759206,7,2020-03-09 00:00:08,2020-03-09 01:07:41,4053
4,15105343836194894915,108,2020-03-09 00:00:10,2020-03-09 00:02:34,144


In [10]:
df_logs.describe

<bound method NDFrame.describe of                       user_id  vsetv_id           start_time  \
0         3398461054087191302         6  2020-03-09 00:00:02   
1        17226860011138219284       353  2020-03-09 00:00:02   
2        18417531283109304442       332  2020-03-09 00:00:05   
3         3677642666679759206         7  2020-03-09 00:00:08   
4        15105343836194894915       108  2020-03-09 00:00:10   
...                       ...       ...                  ...   
5821982  13571199046623737884      1528  2020-07-26 23:58:01   
5821983   2838730117582047512      1174  2020-07-26 23:58:31   
5821984  13614421313501894974         3  2020-07-26 23:58:34   
5821985   7072476312452226836        19  2020-07-26 23:58:39   
5821986  13866173795451203812       403  2020-07-26 23:58:42   

                   stop_time  duraton  
0        2020-03-09 00:01:53      111  
1        2020-03-09 00:03:26      204  
2        2020-03-09 00:01:26       81  
3        2020-03-09 01:07:41     4053

In [11]:
def load_logs(path: str | Path, min_duration_sec: int = 180) -> pd.DataFrame:
    """
    Đọc file logs (dataset11-30.csv), chuẩn hoá cột và lọc lượt xem ngắn.
    - Đổi tên: stop_time->end_time, duraton->duration
    - Parse datetime cho start_time, end_time
    - Lọc duration >= min_duration_sec (min_duration_sec >= 80% program duration)
    """
    df = read_csv(path)
    df = df.rename(columns={"stop_time": "end_time", "duraton": "duration"}).copy()

    # Bắt buộc có các cột tối thiểu
    required = {"user_id", "vsetv_id", "start_time", "end_time", "duration"}
    missing = required - set(df.columns)
    if missing:
        raise ValueError(f"Missing columns in logs: {missing}")

    # Parse datetime cho logs (thường đã ở dạng ISO)
    df["start_time"] = pd.to_datetime(df["start_time"], errors="raise", utc=False)
    df["end_time"]   = pd.to_datetime(df["end_time"],   errors="raise", utc=False)

    # Duration numeric & filter
    df["duration"] = pd.to_numeric(df["duration"], errors="coerce").fillna(0).astype(int)
    df = df[df["duration"] >= int(min_duration_sec)].reset_index(drop=True)

    # Đổi tên cột nhất quán downstream
    df = df.rename(columns={
        "start_time": "start_time_view",
        "end_time":   "end_time_view",
        "duration":   "duration_view",
    })
    return df[["user_id", "vsetv_id", "start_time_view", "end_time_view", "duration_view"]]


In [12]:
log = load_logs(logs)

In [20]:
log.head

<bound method NDFrame.head of                       user_id  vsetv_id     start_time_view  \
0        17226860011138219284       353 2020-03-09 00:00:02   
1         3677642666679759206         7 2020-03-09 00:00:08   
2        18081325105171528512       921 2020-03-09 00:00:11   
3         8504965706466181080      1249 2020-03-09 00:00:13   
4         7072476312452226836       777 2020-03-09 00:00:13   
...                       ...       ...                 ...   
3927541   3473913935630829347        39 2020-07-26 23:55:15   
3927542  10013871288730343419        19 2020-07-26 23:55:17   
3927543   9451216938908035701       921 2020-07-26 23:55:36   
3927544   8639440216155909597       652 2020-07-26 23:55:40   
3927545   5201886213552257351         7 2020-07-26 23:55:49   

              end_time_view  duration_view  
0       2020-03-09 00:03:26            204  
1       2020-03-09 01:07:41           4053  
2       2020-03-09 00:05:41            330  
3       2020-03-09 00:32:58      

# Ghép log ↔ lịch phát sóng theo kênh + khoảng thời gian (interval join)
- logs: mỗi dòng là một lượt xem của user trên một kênh (vsetv_id) với khoảng thời gian [start_time_view, end_time_view].
- meta: lịch phát sóng của cùng kênh, mỗi dòng là một tập/chương trình với khoảng thời gian [start_time, prog_end].
- Mục tiêu: với mỗi lượt xem, tìm một chương trình trong lịch phát sóng trên cùng kênh mà chồng lấn (overlap) với lượt xem lớn nhất, rồi gán tv_show_id cho lượt xem đó.
![Alt text](image.png)

In [13]:
from __future__ import annotations
import argparse
from pathlib import Path
import numpy as np
import pandas as pd

In [14]:
def _pick_overlap(logs_ch: pd.DataFrame, meta_ch: pd.DataFrame) -> pd.DataFrame:
    """
    Với mỗi lượt xem (view) trên 1 kênh:
      - Chẻ view theo ranh giới chương trình trong meta_ch (đã sort theo start_time).
      - Trả về nhiều hàng (segment) cho một view nếu view bắc cầu >1 show.
    Yêu cầu meta_ch có các cột: ["vsetv_id","start_time","prog_end","tv_show_id"]
    và đã lọc tv_show_id != 0 từ trước.
    """
    if logs_ch.empty or meta_ch.empty:
        return pd.DataFrame(columns=[
            "user_id","tv_show_id","vsetv_id",
            "start_time_view","end_time_view","duration_view",
            "seg_start","seg_end","overlap_s","show_duration_s",
            "view_id","seg_idx"
        ])

    # Bảo đảm sort theo start_time để tìm kiếm nhị phân
    meta_ch = meta_ch.sort_values("start_time").reset_index(drop=True)

    # Chuẩn bị mảng nhanh
    # MẢNG NHANH – ép kiểu an toàn
    m_start = meta_ch["start_time"].to_numpy(dtype="datetime64[ns]")
    m_end   = meta_ch["prog_end"].to_numpy(dtype="datetime64[ns]")
    m_tid   = pd.to_numeric(meta_ch["tv_show_id"], errors="coerce").to_numpy()
    m_tid   = np.ravel(m_tid)  # phẳng 1-D

    # Gán view_id
    logs_ch = logs_ch.copy()
    logs_ch["view_id"] = np.arange(len(logs_ch), dtype=np.int64)

    out_rows = []
    for _, row in logs_ch.iterrows():
        v_start = np.datetime64(row["start_time_view"])
        v_end   = np.datetime64(row["end_time_view"])
        if v_end <= v_start:
            continue

        # Tìm show có start_time <= v_start (vị trí bắt đầu duyệt)
        # searchsorted trả vị trí chèn để giữ m_start tăng dần
        # side='right' rồi -1 để lấy phần tử <= v_start
        i = np.searchsorted(m_start, v_start, side="right") - 1
        if i < 0:
            i = 0

        seg_idx = 0
        # Duyệt các show cho đến khi start_time_show >= v_end
        # (vì những show bắt đầu sau v_end sẽ không thể overlap)
        n = len(meta_ch)
        while i < n and m_start[i] < v_end:
            s_show = m_start[i]
            e_show = m_end[i]
            # Nếu show kết thúc trước v_start thì nhảy tiếp
            if e_show <= v_start:
                i += 1
                continue
            # Nếu show bắt đầu sau v_end thì dừng
            if s_show >= v_end:
                break

            # Tính đoạn overlap giữa view và show
            seg_start = max(v_start, s_show)
            seg_end   = min(v_end, e_show)
            if seg_end > seg_start:
                overlap_s = int((seg_end - seg_start) / np.timedelta64(1, "s"))
                show_duration_s = int((e_show - s_show) / np.timedelta64(1, "s"))

                # LẤY tv_show_id an toàn
                tid_val = m_tid[i]
                if isinstance(tid_val, (np.ndarray, list)):
                    tid_val = np.ravel(tid_val)[0]
                if pd.isna(tid_val):
                    i += 1
                    continue
                tid_val = int(tid_val)
            
                out_rows.append({
                    "user_id": row["user_id"],
                    "tv_show_id": int(m_tid[i]),
                    "vsetv_id": row["vsetv_id"],
                    "start_time_view": row["start_time_view"],
                    "end_time_view": row["end_time_view"],
                    "duration_view": row["duration_view"],
                    "seg_start": pd.Timestamp(seg_start),
                    "seg_end": pd.Timestamp(seg_end),
                    "overlap_s": overlap_s,
                    "show_duration_s": show_duration_s,
                    "view_id": row["view_id"],
                    "seg_idx": seg_idx,
                })
                seg_idx += 1

            # Sang show kế tiếp
            i += 1

    if not out_rows:
        return pd.DataFrame(columns=[
            "user_id","tv_show_id","vsetv_id",
            "start_time_view","end_time_view","duration_view",
            "seg_start","seg_end","overlap_s","show_duration_s",
            "view_id","seg_idx"
        ])
    return pd.DataFrame(out_rows)

def interval_join_logs_with_program(logs: pd.DataFrame, meta: pd.DataFrame) -> pd.DataFrame:
    out = []
    for vid, logs_ch in logs.groupby("vsetv_id", sort=False):
        meta_ch = meta[meta["vsetv_id"] == vid]
        matched = _pick_overlap(logs_ch, meta_ch)
        if not matched.empty:
            out.append(matched)
    if not out:
        return pd.DataFrame(columns=[
            "user_id","tv_show_id","vsetv_id",
            "start_time_view","end_time_view","duration_view",
            "seg_start","seg_end","overlap_s","show_duration_s",
            "view_id","seg_idx"
        ])
    return pd.concat(out, ignore_index=True)


In [15]:
joined = interval_join_logs_with_program(log, meta)

In [16]:
joined.head

<bound method NDFrame.head of               user_id  tv_show_id  vsetv_id     start_time_view  \
0        5.735792e+16     6600437       353 2020-03-09 07:39:35   
1        8.404698e+18     6600437       353 2020-03-09 07:40:45   
2        1.056175e+19     6600437       353 2020-03-09 07:40:48   
3        5.102445e+18     6600437       353 2020-03-09 07:48:30   
4        1.725563e+18     6600437       353 2020-03-09 07:54:15   
...               ...         ...       ...                 ...   
2990102  4.234566e+18    12002809        54 2020-07-26 22:40:52   
2990103  7.457127e+18    12002809        54 2020-07-26 22:49:01   
2990104  1.600956e+19    12002352        54 2020-07-26 23:30:01   
2990105  1.600956e+19    12002352        54 2020-07-26 23:34:15   
2990106  1.600956e+19    12002352        54 2020-07-26 23:34:15   

              end_time_view  duration_view           seg_start  \
0       2020-03-09 07:44:41            306 2020-03-09 07:39:35   
1       2020-03-09 08:06:41      

In [17]:
# joined["screen_time"] = joined["overlap_s"] / joined["show_duration_s"]
# eligible = joined[joined["screen_time"] >= 0.8].copy()

In [18]:
joined["screen_time"] = joined["overlap_s"] / joined["show_duration_s"]

eligible = joined[joined["screen_time"] >= 0.8].copy()


In [19]:
eligible.head

<bound method NDFrame.head of               user_id  tv_show_id  vsetv_id     start_time_view  \
52       1.120433e+19     6600437       353 2020-03-09 20:11:48   
332      1.362266e+18     6600437       353 2020-03-13 01:15:15   
401      2.167424e+18     6600437       353 2020-03-14 07:37:17   
511      1.215370e+19     6600437       353 2020-03-15 01:42:35   
553      1.818677e+17     6600437       353 2020-03-15 16:20:52   
...               ...         ...       ...                 ...   
2990092  1.516048e+18    12002262        54 2020-07-26 20:40:15   
2990094  1.978724e+18    12002262        54 2020-07-26 20:40:53   
2990096  1.226811e+19    12002809        54 2020-07-26 22:13:57   
2990097  1.226811e+19    12002809        54 2020-07-26 22:13:57   
2990098  1.226811e+19    12002809        54 2020-07-26 22:13:57   

              end_time_view  duration_view           seg_start  \
52      2020-03-09 22:12:23           7235 2020-03-09 20:11:48   
332     2020-03-13 03:58:40      